In [4]:
f = open(r'C:\Users\CZY\Desktop\AI_project3\cifar-10-batches-py/batches.meta', mode='r')

In [16]:
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Activation,Flatten,BatchNormalization
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pickle

def unpickle(file):
      with open(file, 'rb') as fo:
              dict = pickle.load(fo, encoding='bytes')
      return dict

In [7]:
# 测试集文件
test_batch = unpickle(R'C:\Users\CZY\Desktop\AI-3\cifar-10-batches-py/test_batch')

X_test = np.array(test_batch[b'data'])
y_test = np.array(test_batch[b'labels']) 

In [8]:
# 训练集文件
train_files = ['data_batch_'+str(i) for i in range(1,6)]

train_data_list,train_label_list = [],[]

for f in train_files:
    
    fpath = r'C:\Users\CZY\Desktop\AI-3\cifar-10-batches-py/' + f
    batch_dict = unpickle(fpath)
    
    batch_data = batch_dict[b'data']
    batch_labels = batch_dict[b'labels']
    train_data_list.append(batch_data)
    train_label_list.append(batch_labels)

X_train = np.concatenate(train_data_list, axis = 0)
y_train = np.concatenate(train_label_list, axis = 0)

In [9]:
label_names_batch = unpickle(r'C:\Users\CZY\Desktop\AI-3\cifar-10-batches-py/batches.meta')
label_names = label_names_batch[b'label_names']
label_names = [l.decode("utf-8") for l in label_names]

In [10]:
print('训练集特征：',X_train.shape,'，训练集label',y_train.shape)
print('测试集特征：',X_test.shape,'，测试集label', y_test.shape)
print("类别名字：",label_names)

训练集特征： (50000, 3072) ，训练集label (50000,)
测试集特征： (10000, 3072) ，测试集label (10000,)
类别名字： ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [11]:
X_train = X_train.reshape(-1,3,32,32)
X_test = X_test.reshape(-1,3,32,32)

In [12]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 归一化
X_train /= 255
X_test /= 255

# 将class vectors转变成binary class metrics
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [13]:
# Method 1: Switch backend
keras.backend.set_image_dim_ordering('th')
print('Backend:',keras.backend.image_dim_ordering())


Backend: th


In [14]:
def base_model(opt):
    model = Sequential()
    model.add(Conv2D(32,(3,3), padding="same", input_shape=X_train.shape[1:]))
    model.add(Activation('relu')
    MaxPooling2D(pool_size=(2, 2), data_format="channels_first")
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64,(3,3),padding="same"))
    model.add(Activation('relu'))
    MaxPooling2D(pool_size=(2, 2), data_format="channels_first")
    model.add(Dropout(0.25))
    model.add(Flatten())   
    model.add(Dropout(0.25))
    model.add(Dense(512))  
    model.add(Activation('relu'))
    
    model.add(Dropout(0.25))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) 
    return model

In [17]:
opt1 = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
cnn2 = base_model(opt1)
cnn2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 32, 32)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 64, 32, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 64, 32, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
__________

In [18]:
epochs = 12
batch_size = 100
verbose = True

In [58]:
history = cnn2.fit(X_train, y_train, 
                    epochs = 10, batch_size = 32, 
                    validation_data=(X_test,y_test), 
                    shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 567s 11ms/step - loss: 1.6415 - acc: 0.4121 - val_loss: 1.3445 - val_acc: 0.5297
Epoch 2/10
50000/50000 [==============================] - 539s 11ms/step - loss: 1.3009 - acc: 0.5399 - val_loss: 1.2011 - val_acc: 0.5825
Epoch 3/10
50000/50000 [==============================] - 537s 11ms/step - loss: 1.1619 - acc: 0.5935 - val_loss: 1.1212 - val_acc: 0.6111
Epoch 4/10
50000/50000 [==============================] - 538s 11ms/step - loss: 1.0627 - acc: 0.6307 - val_loss: 1.0775 - val_acc: 0.6286
Epoch 5/10
50000/50000 [==============================] - 538s 11ms/step - loss: 0.9881 - acc: 0.6567 - val_loss: 1.0305 - val_acc: 0.6396
Epoch 6/10
50000/50000 [==============================] - 537s 11ms/step - loss: 0.9255 - acc: 0.6820 - val_loss: 1.0176 - val_acc: 0.6525
Epoch 7/10
50000/50000 [==============================] - 538s 11ms/